<font color='red'><b>EJERCICIO 1:<br/>
El objetivo es programar una función que reciba como input un texto de usuario y devuelva los fragmentos
de texto (chunks) que hagan referencia a las comidas y cantidades que ha solicitado. No es necesario, ni es
el objeti<font color='red'>vo de este ejercicio, construir un clasificador de intención previo a esta función, sino simplemente una
función que presuponemos recibe una frase con la intención 'Pedir_comida'. Tampoco es objetivo normalizar
la salida (por ej.: no es necesario convertir 'tres' a '3' ni 'pizzas' a 'pizza'). Es, por tanto, un ejercicio de
mínimos.

Por ejemplo: “quiero 3 bocadillos de anchoas y 2 pizzas” →
{comida:'bocadillo', ingrediente:'anchoas', cantidad:3},
{comida:'pizza', ingrediente:'null', cantidad:2}</b></font>

In [2]:
"""
Analizador de envios de comida
"""
from nltk.corpus import cess_esp
import nltk
from nltk import UnigramTagger, BigramTagger, TrigramTagger
import re
from nltk.collections import LazyMap

<b>Lo primero que hacemos es crear un corpus de frases, mi objetivo en esta práctica es un objetivo de mínimos. El programa resultante se centrará en recoger una cantidad, una comida y un ingrediente de cada frase, aunque haya frases con mas de un ingrediente. No es el objetivo extraerlo todo, no obstante si que hay frases con varios ingredientes y veremos que el tagging y el chunking se realizan bien aunque la función extractora final no lo hace.</b>

In [3]:
"""*****************************************************
1. Se crea un corpus de frases de ejemplo.
*****************************************************"""
corpus = ["Buenos dias, quiero comprar 3 pizzas con anchoas, diez bocadillos de salchichón y 15 perritos con mostaza.", 
          "Hola, voy a pedir 55 perritos con mostaza.",
          "Mi pedido es: dos hamburguesas de pollo.",
          "Voy a comprar tres pizzas con piña.",
          "Buenos dias, quiero cuatro perritos con queso.",
          "Hola, necesito cinco pizzas familiares con jamon,gracias",
          "Pediré diez hamburguesas con bacon.",
          "Vamos a pedir un bocadillo de chorizo.",
          "Vamos a querer una pizza con orégano y salchichon.",
          "¿Podemos pedir seis hamburguesas con huevo? Gracias",
          "Me apetecen nueve bocadillos de bacon y queso y seis pizzas familiares con piña y chorizo.",
          "Quiero tres pizzas",
          "Yo quería una hamburguesa"]

<b>Vamos a entrenar con frases del corpus <font color='blue'>cess_esp</font> un etiquetador TrigramTagger para después usarlo en nuestro corpus de frases. Vamos a cargar el corpus para el español y seleccionamos un conjunto de entrenamiento (90%)  y test(10%)
    </b>

In [4]:
"""**************************************************
2. Después, se debe entrenar un tagger para el español (es necesario
consultar la práctica 2 de NLTK en la que usábamos
el cess esp para obtener los tags de un corpus en español).
*****************************************************"""
sents = cess_esp.tagged_sents()

training = []
test = []
for i in range(len(sents)):
    if i % 10:
        training.append(sents[i])
    else:
        test.append(sents[i])

<b>El conjunto de POS TAGs en español es el <a href=https://www.cs.upc.edu/~nlp/tools/parole-sp.html>EAGLE</a></b>
<b>    
    Esta es una página donde se muestra un subonjunto de los POS tags con los que se van a etiquetar los tokens:</b><br/>
<a href="https://nlp.stanford.edu/software/spanish-faq.shtml#tagset">STANDFORD SPANISH TAGSET</a>

<b>
    Para el tagging vamos a usar <font color='blue'>TrigramTagger</font> pero usando el método backoff y que se ayude de <font color='blue'>BigramTagger</font> , 
    este a su vez de <font color='blue'>UnigramTagger</font> y este a su vez de un <font color='blue'>RegexpTagger </font>, al cual le proporcionamos una serie de 
patrones determinados para etiquetar la comida , incluso ponemos un patrón final (r’.*,’nc0000’) para etiquetar 
cualquier token que no haya podido ser reconocido. 
He hecho este RegexpTagger porque  los otros tagger no eran capaces de reconocer ninguna comida y es un objetivo 
clave de este ejercicio. Estoy suponiendo que la empresa de comida tiene un menú de comidas determinado fijado 
por la lista que aparece en este patrón:</b>
<table>
    <tr><th>Comida</th><th>Ingredientes</th></tr>
    <tr><td>bocadillo</td><td>Salchichón</td></tr>
    <tr><td>pizza</td><td>jamón</td></tr>
    <tr><td>hamburguesa</td><td>orégano</td></tr>
    <tr><td>perrito</td><td>chorizo</td></tr>
    <tr><td/><td>anchoas</td></tr>
    <tr><td/><td>piña</td></tr>
    <tr><td/><td>cebolla</td></tr>
    <tr><td/><td>queso</td></tr>
    </table>

In [5]:
patrones = [
(r'bocadillo[s]?', 'nc0p000'),
(r'pizza[s]?', 'nc0p000'),
(r'hamburguesa[s]?', 'nc0p000'),
(r'perrito[s]?', 'nc0p000'),
(r'salchich[o|ó]n', 'ncfs000'),
(r'jam[o|ó]n', 'ncfs000'),
(r'or[e|é]gano', 'ncfs000'),
(r'chorizo', 'ncfs000'),
(r'anchoa[s]?', 'ncfs000'),
(r'piña', 'ncfs000'),
(r'cebolla', 'ncfs000'),
(r'queso', 'ncfs000'),
(r'.*', 'nc00000')]

<b>Realizamos el entrenamiento de nuestro <font color='blue'>spanish_pos_tagger</font>:</b>

In [6]:
regexp_tagger = nltk.RegexpTagger(patrones)
unigram_tagger = UnigramTagger(training,backoff=regexp_tagger)
bigram_tagger = BigramTagger(training, backoff=unigram_tagger)
spanish_pos_tagger = TrigramTagger(training, backoff=bigram_tagger)

<b>Ahora vamos a dividir las frases en tokens y a hacer que nuestro tagger etiquete cada token de cada frase con el spanish_pos_tagger definido y entrenado anteriormente.<br/>
Imprimimos cada frase en la que puede verse que cada token aparece con el POS tag asignado:</b>

In [7]:
for frase in corpus:
    print ("_________________________________________________________________")
    print(frase)
    tokens = nltk.word_tokenize(frase)
    tagged = spanish_pos_tagger.tag(tokens)
    print ("SPANISH TAGGER:",tagged)
    print ("_________________________________________________________________")

_________________________________________________________________
Buenos dias, quiero comprar 3 pizzas con anchoas, diez bocadillos de salchichón y 15 perritos con mostaza.
SPANISH TAGGER: [('Buenos', 'nc00000'), ('dias', 'nc00000'), (',', 'Fc'), ('quiero', 'vmip1s0'), ('comprar', 'vmn0000'), ('3', 'Z'), ('pizzas', 'nc0p000'), ('con', 'sps00'), ('anchoas', 'ncfs000'), (',', 'Fc'), ('diez', 'dn0cp0'), ('bocadillos', 'nc0p000'), ('de', 'sps00'), ('salchichón', 'ncfs000'), ('y', 'cc'), ('15', 'Z'), ('perritos', 'nc0p000'), ('con', 'sps00'), ('mostaza', 'ncfs000'), ('.', 'Fp')]
_________________________________________________________________
_________________________________________________________________
Hola, voy a pedir 55 perritos con mostaza.
SPANISH TAGGER: [('Hola', 'nc00000'), (',', 'Fc'), ('voy', 'vmip1s0'), ('a', 'sps00'), ('pedir', 'vmn0000'), ('55', 'Z'), ('perritos', 'nc0p000'), ('con', 'sps00'), ('mostaza', 'ncfs000'), ('.', 'Fp')]
__________________________________________

<b>Evaluamos al spanish_pos_tagger:</b>

In [8]:
print ('Acierto con spanish_pos_tagger:', 
       spanish_pos_tagger.evaluate(training)*100)

Acierto con spanish_pos_tagger: 99.07466055531883


<b>
CHUNKING<br/>
    Ahora vamos a diseñar un <font color='blue'>RegexpParser</font> que detecte comida, cantidades e Ingredientes, todo ello dentro de un pedido:
Con las siguientes premisas:
<font color='green'>
    <ol>
        <li>- Un pedido es una comida mas unos ingredientes.</li>
        <li>- La comida es el nombre de esta y la cantidad</li>
        <li>- La cantidad puede ser numérica o no.</li>
        <li>- Los ingredientes son cualquier nombre que acompañe a la comida.</li>
</font>
    </ol>
</b>

In [9]:
gramatica=r"""
    Cantidad: {<Z|dn.*|di0.*|pn.*>}
    Comida: {<Cantidad><nc.*>}
    Ingredientes: {<sps00><nc.*>(<cc><nc.*>)*}
    Pedido: {<Comida>(<.*>)?<Ingredientes>*}"""
regex_parser = nltk.RegexpParser(gramatica)

<b>y la probamos con cada frase del corpus ya tokenizada y etiquetada.<br/>
Aquí está el resultado para las frases del corpus:
</b>

In [10]:
"""**************************************************
4. Se usa el pos tagger y el Regex Parser para obtener las IOB de una nueva
frase.
*****************************************************"""

for frase in corpus:
    print(frase)
    tokens = nltk.word_tokenize(frase)
    tagged = spanish_pos_tagger.tag(tokens)
    result = regex_parser.parse(tagged)
    print(result)
    print ("_________________________________________________________________")

Buenos dias, quiero comprar 3 pizzas con anchoas, diez bocadillos de salchichón y 15 perritos con mostaza.
(S
  Buenos/nc00000
  dias/nc00000
  ,/Fc
  quiero/vmip1s0
  comprar/vmn0000
  (Pedido
    (Comida (Cantidad 3/Z) pizzas/nc0p000)
    (Ingredientes con/sps00 anchoas/ncfs000))
  ,/Fc
  (Pedido
    (Comida (Cantidad diez/dn0cp0) bocadillos/nc0p000)
    (Ingredientes de/sps00 salchichón/ncfs000))
  y/cc
  (Pedido
    (Comida (Cantidad 15/Z) perritos/nc0p000)
    (Ingredientes con/sps00 mostaza/ncfs000))
  ./Fp)
_________________________________________________________________
Hola, voy a pedir 55 perritos con mostaza.
(S
  Hola/nc00000
  ,/Fc
  voy/vmip1s0
  a/sps00
  pedir/vmn0000
  (Pedido
    (Comida (Cantidad 55/Z) perritos/nc0p000)
    (Ingredientes con/sps00 mostaza/ncfs000))
  ./Fp)
_________________________________________________________________
Mi pedido es: dos hamburguesas de pollo.
(S
  Mi/dp1css
  pedido/vmp00sm
  es/vsip3s0
  :/Fd
  (Pedido
    (Comida (Cantidad dos/d

<b>
Una vez que tenemos nuestros arboles nltk para cada frase, definimos unas funciones para recorrer estos arboles identificando nuestros chunks (pedido, comida, cantidad e ingredientes) y extrayendo la información (no he sido absolutamente exhaustivo… por ejemplo solo se extrae un ingrediente).<br/>
    La función <font color='blue'>recorrer_arbol(tree,nombreNodo,lista)</font> recorre (primero en profundidad) recursivamente el arbol nltk asociado a una frase hasta que encuentra un nodo con el nombre del parámetro <i>nombreNodo<i/> y entonces añade los hijos del nodo a la lista. Así por ejemplo si llamamos a recorrer_arbol(arbol,"Cantidad",lista) nos devolverá una lista con los hijos del nodo "Cantidad" , es decir un lista así: [('3', 'Z')] <br/>
    
La función <font color='blue'>pedir_Comida(arbolpedido)</font> recorre un arbol nltk para cada etiqueta que nos interesa (Cantidad,Comida,Ingredientes) y formatea la salida para devolver un diccionario
</b>

In [11]:
def recorrer_arbol(tree,nombreNodo,lista):
    #print("tree:", tree.label())
    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            recorrer_arbol(subtree,nombreNodo,lista)
        else:
            if tree.label() == nombreNodo:
                #print ("Word:", subtree[0])
                lista.append(subtree)
      

def pedir_Comida(arbolpedido):
    listacomida=[]   
    listacantidad=[]       
    listaingredientes=[]  
    recorrer_arbol(arbolpedido,'Comida',listacomida)
    #print(listacomida)
    recorrer_arbol(arbolpedido,'Cantidad',listacantidad)
    #print(listacantidad)
    recorrer_arbol(arbolpedido,'Ingredientes',listaingredientes)
    patron = re.compile('nc.*')
    listaingredientes = [ingred[0] for ingred in listaingredientes 
                        if patron.match(ingred[1])]
    #print(listaingredientes)
    
    if len(listaingredientes)>0:
        return([dict(cantidad=cant[0],comida=comi[0],ingredientes=ingred)   
              for cant, comi,ingred  
              in zip(listacantidad, listacomida,listaingredientes)])
    else:
        return([dict(cantidad=cant[0],comida=comi[0],ingredientes='null')   
              for cant, comi  
              in zip(listacantidad, listacomida)])

<b>
    Podemos ver el resultado final llamando a <font color='blue'>pedir_Comida()</font> para cada frase del corpus:
</b>

In [12]:
lista_chunks = []
for frase in corpus:
    print(frase)
    tokens = nltk.word_tokenize(frase)
    tagged = spanish_pos_tagger.tag(tokens)
    result = regex_parser.parse(tagged)
    #print(result)
    print(pedir_Comida(result))
    lista_chunks.append(result)
    print ("_________________________________________________________________")

Buenos dias, quiero comprar 3 pizzas con anchoas, diez bocadillos de salchichón y 15 perritos con mostaza.
[{'cantidad': '3', 'comida': 'pizzas', 'ingredientes': 'anchoas'}, {'cantidad': 'diez', 'comida': 'bocadillos', 'ingredientes': 'salchichón'}, {'cantidad': '15', 'comida': 'perritos', 'ingredientes': 'mostaza'}]
_________________________________________________________________
Hola, voy a pedir 55 perritos con mostaza.
[{'cantidad': '55', 'comida': 'perritos', 'ingredientes': 'mostaza'}]
_________________________________________________________________
Mi pedido es: dos hamburguesas de pollo.
[{'cantidad': 'dos', 'comida': 'hamburguesas', 'ingredientes': 'pollo'}]
_________________________________________________________________
Voy a comprar tres pizzas con piña.
[{'cantidad': 'tres', 'comida': 'pizzas', 'ingredientes': 'piña'}]
_________________________________________________________________
Buenos dias, quiero cuatro perritos con queso.
[{'cantidad': 'cuatro', 'comida': 'perri

In [13]:
print(regex_parser.evaluate(lista_chunks))

ChunkParse score:
    IOB Accuracy: 100.0%%
    Precision:    100.0%%
    Recall:       100.0%%
    F-Measure:    100.0%%


<font color='red'><b>EJERCICIO 2<br/>
    Analizador de envios de comida con UnigramTagger y BigramTagger
</b></font>

<b>Lo primero que hacemos es crear un fichero de entrenamiento llamado <font color='blue'>corpus_train.txt</font> de frases con POS tags y con chunk tags (IOB) . Los  POS tags que he usado son los mismos que los del ejercicio 1 , los que tiene el corpus cess_esp:
<a href=https://www.cs.upc.edu/~nlp/tools/parole-sp.html>EAGLE</a><br/>
Este es el fichero:

<b>
El fichero está confeccionado siguiendo las siguientes características:<br/>
    <font color='green'><ol>
        <li>En el fichero, cada token de cada frase esta en una linea y está acompañado de una etiqueta "POS" y de una etiqueta de chunking, en mi caso además de las {I,O, B} etiquetamos con {Cantidad,Comida,Ingredientes} </li>
        <li>Cada frase se separa de la anterior por una linea en blanco.</li>
    </ol></font><br/>
    Defino una función <font color='blue'>lee_frases(nombre_fichero)</font>  para leer los tokens de cada frase atendiendo a las características anteriormente mencionadas:
</b>

In [14]:
def lee_frases(nombre_fichero):
    frase = ""
    frases=[]
    f = open(nombre_fichero,'r',encoding="utf-8")
    for line in f:
        if line.strip():
            frase+=line
        else:
            frases.append(frase)
            frase = ""    
    return frases

<b>Empezamos leyendo el fichero:</b>

In [15]:
ruta_ficheros = '/home/fermasolar/Documentos/MODULO04/Ejercicios/'
corpus_train = lee_frases(ruta_ficheros + '/corpus_train.txt')
print("Número de frases en el corpus de entrenamiento:" , len(corpus_train))
print ("_________________________________________________________________")
print(corpus_train)


Número de frases en el corpus de entrenamiento: 8
_________________________________________________________________
['Buenos nc00000 B\ndias nc00000 I\n, Fc O\nquiero vmip1s0 O\ncomprar vmip1s0 O\n3 Z I-Cantidad\npizzas nc0p000 I-Comida\ncon sps00 O\nanchoas ncfs000 I-Ingredientes\n, Fc O\ndiez dn0cp0 I-Cantidad\nbocadillos nc0p000 I-Comida\nde sps00 O\nsalchichón ncfs000 I-Ingredientes\ny cc O\n15 Z I-Cantidad\nperritos nc0p000 I-Comida\ncon sps00 O\nmostaza ncfs000 I-Ingredientes\n. Fp O\n', 'Hola nc00000 B\n, Fc O\nvoy vmip1s0 O\na sps00 O\npedir vmn0000 O\n55 Z I-Cantidad\nperritos nc0p000 I-Comida\ncon sps00 O\nmostaza ncfs000 I-Ingredientes\n. Fp O\n', 'Mi dp1css B\npedido vmp00sm I\nes vsip3s0 I\n: Fd B\ndos dn0cp0 I-Cantidad\nhamburguesas nc0p000 I-Comida\nde sps00 O\npollo ncms000 I-Ingredientes\n. Fp O\n', 'Voy vmip1s0 B\na sps00 I\ncomprar vmn0000 O\ntres dn0cp0 I-Cantidad\npizzas nc0p000 I-Comida\ncon sps00 O\npiña ncfs000 I-Ingredientes\n. Fp O\n', 'Buenos nc00000 B\ndias 

<b>
    Puesto que voy a diseñar un chunker que use <font color='blue'>UnigramTagger</font> y <font color='blue'>BigramTagger</font> he modificado ligeramene la funcion “”UnigramChunker” del libro <font color='grey'>“Natural Language Processing with Python”</font> y he creado la clase <font color='blue'>NgramChunker</font> con el parámetro  <i>tagger_classes</i> que puede tomar los valores ‘UnigramTagger’ y ‘BigramTagger’ para así poder hacerlo con los dos tagger:
</b>

In [16]:
class NgramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents,tagger_classes ='UnigramTagger'):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        if tagger_classes == 'UnigramTagger':
            self.tagger = nltk.UnigramTagger(train_data)  
        if tagger_classes == 'BigramTagger':
            self.tagger = nltk.BigramTagger(train_data,backoff=
                                            nltk.UnigramTagger(train_data))
        
    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

<b>La función <font color='grey'>“tree2conlltags”</font> requiere que las sentencias estén almacenadas en un objeto del tipo <font color='green'>LazyMap</font>, y por eso y porque yo estoy usando unos determinados chunk tags <font color='blue'>(Comida,Cantidad,Ingredientes)</font> he definido una función para indicarlos y para que su salida sea un LazyMap:</b>

In [17]:
def mifuncion(x):
    tipos=['Cantidad','Comida','Ingredientes']
    return nltk.chunk.conllstr2tree(x,chunk_types=tipos)

<b>Así se ve el objeto <i>LazyMap</i> para las frases del fichero corpus_train.txt:</b>

In [18]:
train_sents=LazyMap(mifuncion, corpus_train)
print(train_sents)

[Tree('S', [('Buenos', 'nc00000'), ('dias', 'nc00000'), (',', 'Fc'), ('quiero', 'vmip1s0'), ('comprar', 'vmip1s0'), Tree('Cantidad', [('3', 'Z')]), Tree('Comida', [('pizzas', 'nc0p000')]), ('con', 'sps00'), Tree('Ingredientes', [('anchoas', 'ncfs000')]), (',', 'Fc'), Tree('Cantidad', [('diez', 'dn0cp0')]), Tree('Comida', [('bocadillos', 'nc0p000')]), ('de', 'sps00'), Tree('Ingredientes', [('salchichón', 'ncfs000')]), ('y', 'cc'), Tree('Cantidad', [('15', 'Z')]), Tree('Comida', [('perritos', 'nc0p000')]), ('con', 'sps00'), Tree('Ingredientes', [('mostaza', 'ncfs000')]), ('.', 'Fp')]), Tree('S', [('Hola', 'nc00000'), (',', 'Fc'), ('voy', 'vmip1s0'), ('a', 'sps00'), ('pedir', 'vmn0000'), Tree('Cantidad', [('55', 'Z')]), Tree('Comida', [('perritos', 'nc0p000')]), ('con', 'sps00'), Tree('Ingredientes', [('mostaza', 'ncfs000')]), ('.', 'Fp')]), ...]


<b>
    Con ello , ya puedo llamar a <font color='blue'>NgramChunker</font> para el corpus de entrenamiento<br/>
    Por defecto se hace con la clase UnigramTagger para el POS tagging:
</b>

In [27]:
unigram_chunker = NgramChunker(train_sents,tagger_classes='UnigramTagger')

<b>
    ¡¡¡Vamos a probar el chunker!!<br/>
    He creado un fichero de frases de test llamado <font color='blue'>corpus_test.txt</font>. <br/>
    Este es el fichero:
    </b>

<b>
    Creo la función <font color='blue'>lee_fichero(nombre_fichero)</font> para leer frases de un fichero de texto organizado de forma que en cada linea hay un frase:
    </b>

In [20]:
def lee_fichero(nombre_fichero):
    frases=[]
    f = open(nombre_fichero,'r',encoding="utf-8")
    for line in f.readlines():
        frases.append(line)
    return frases

<b>
    Leemos el fichero de test:
    </b>

In [21]:
corpus_test=lee_fichero(ruta_ficheros + 'corpus_test.txt')
print(corpus_test)

['Vamos a querer una pizza con orégano.\n', '¿Podemos pedir 6 hamburguesas con huevo? Gracias\n', 'Me apetecen 9 bocadillos de bacon y 6 pizzas familiares con chorizo.\n', 'Quiero 3 pizzas\n', 'Yo quería una hamburguesa\n']


<b>
    Las frases del corpus de test no están ni tokenizadas ni tienen POS tags , para hacerlo usamos el código del ejercicio 1 (usando RegexpTagger,UnigramTagger,BigramTagger y TrigramTagger):<br/>
    Esta es la lista de frases de test con sus  POS tags:
    </b>

In [22]:
tagged_frases = []
for frase in corpus_test:
    tokens = nltk.word_tokenize(frase)
    tagged = spanish_pos_tagger.tag(tokens)
    tagged_frases.append(tagged)
print(tagged_frases)

[[('Vamos', 'vmip1p0'), ('a', 'sps00'), ('querer', 'vmn0000'), ('una', 'di0fs0'), ('pizza', 'nc0p000'), ('con', 'sps00'), ('orégano', 'ncfs000'), ('.', 'Fp')], [('¿Podemos', 'nc00000'), ('pedir', 'vmn0000'), ('6', 'Z'), ('hamburguesas', 'nc0p000'), ('con', 'sps00'), ('huevo', 'ncms000'), ('?', 'Fit'), ('Gracias', 'nc00000')], [('Me', 'pp1cs000'), ('apetecen', 'nc00000'), ('9', 'Z'), ('bocadillos', 'nc0p000'), ('de', 'sps00'), ('bacon', 'ncmn000'), ('y', 'cc'), ('6', 'Z'), ('pizzas', 'nc0p000'), ('familiares', 'ncmp000'), ('con', 'sps00'), ('chorizo', 'ncfs000'), ('.', 'Fp')], [('Quiero', 'vmip1s0'), ('3', 'Z'), ('pizzas', 'nc0p000')], [('Yo', 'pp1csn00'), ('quería', 'vmii1s0'), ('una', 'di0fs0'), ('hamburguesa', 'nc0p000')]]


<b>
Voy a modificar ligeramente la función pedir_comida haciendo una segunda versión<br/>
    mas adaptada a este ejercicio (y al siguiente ejercicio 3)<br/>
    Llamaré a esta versión <font color='blue'>pedir_Comida2(arbolpedido)</font>:
</b>

In [24]:
def pedir_Comida2(arbolpedido):
    listacomida=[]   
    listacantidad=[]       
    listaingredientes=[]  
    recorrer_arbol(arbolpedido,'Comida',listacomida)
    #print("COMIDA-->" ,listacomida)
    recorrer_arbol(arbolpedido,'Cantidad',listacantidad)
    #print("CANTIDAD-->",listacantidad)
    recorrer_arbol(arbolpedido,'Ingredientes',listaingredientes)
    #print("INGREDIENTES-->",listaingredientes)
    
    if len(listacantidad)>0:
        if len(listaingredientes)>0:
            return([dict(cantidad=cant[0],comida=comi[0],ingredientes=ingred[0])   
              for cant, comi,ingred  
              in zip(listacantidad, listacomida,listaingredientes)])
        else:
            return([dict(cantidad=cant[0],comida=comi[0],ingredientes='null')   
                for cant, comi
                    in zip(listacantidad, listacomida)])
    else:
        if len(listaingredientes)>0:
            return([dict(cantidad='null',comida=comi[0],ingredientes=ingred[0])   
              for comi, ingred  
              in zip(listacomida,listaingredientes)])
        else:
            return([dict(cantidad='null',comida=comi[0],ingredientes='null')   
                  for comi in listacomida])

<b>CHUNKING<br/>
    Y llegamos a la parte clave de esta segunda práctica. Para cada frase del corpus de test,  llamo al método <font color='blue'>“parse”</font> de la clase <font color='blue'><i>NgramChunker</i></font>,  el cual genera un árbol nltk al cual le aplico la función <font color='green'>“pedir_Comida2()”</font> para extraer de ese árbol las partes que me interesan, es decir , el contenido de los chunks “comida,cantidad,ingredientes” :
    </b>

In [28]:
listafrasestest = []
for frasetest in tagged_frases:
    arbol = unigram_chunker.parse(frasetest)
    #print(arbol)
    listafrasestest.append(arbol)
    print(pedir_Comida2(arbol))

[{'cantidad': 'null', 'comida': 'pizza', 'ingredientes': 'orégano'}]
[{'cantidad': '6', 'comida': 'hamburguesas', 'ingredientes': 'huevo'}]
[{'cantidad': '9', 'comida': 'bocadillos', 'ingredientes': 'bacon'}, {'cantidad': '6', 'comida': 'pizzas', 'ingredientes': 'chorizo'}]
[{'cantidad': '3', 'comida': 'pizzas', 'ingredientes': 'null'}]
[{'cantidad': 'null', 'comida': 'hamburguesa', 'ingredientes': 'null'}]


In [29]:
score = unigram_chunker.evaluate(listafrasestest)
print(score)

ChunkParse score:
    IOB Accuracy: 100.0%%
    Precision:    100.0%%
    Recall:       100.0%%
    F-Measure:    100.0%%


<b>
    Voy a probar con <font color='blue'>NgramChunker</font> pero ahora usando <i>"BigramTagger"</i> para las etiquetas POS en el entrenamiento:
</b>

In [30]:
bigram_chunker = NgramChunker(train_sents,tagger_classes='BigramTagger')
listafrasestest = []
for frasetest in tagged_frases:
    arbol = bigram_chunker.parse(frasetest)
    print(arbol)
    listafrasestest.append(arbol)
    print(pedir_Comida2(arbol))

(S
  Vamos/vmip1p0
  a/sps00
  querer/vmn0000
  una/di0fs0
  (Comida pizza/nc0p000)
  con/sps00
  (Ingredientes orégano/ncfs000)
  ./Fp)
[{'cantidad': 'null', 'comida': 'pizza', 'ingredientes': 'orégano'}]
(S
  ¿Podemos/nc00000
  pedir/vmn0000
  (Cantidad 6/Z)
  (Comida hamburguesas/nc0p000)
  con/sps00
  (Ingredientes huevo/ncms000)
  ?/Fit
  Gracias/nc00000)
[{'cantidad': '6', 'comida': 'hamburguesas', 'ingredientes': 'huevo'}]
(S
  Me/pp1cs000
  apetecen/nc00000
  (Cantidad 9/Z)
  (Comida bocadillos/nc0p000)
  de/sps00
  (Ingredientes bacon/ncmn000)
  y/cc
  (Cantidad 6/Z)
  (Comida pizzas/nc0p000)
  familiares/ncmp000
  con/sps00
  (Ingredientes chorizo/ncfs000)
  ./Fp)
[{'cantidad': '9', 'comida': 'bocadillos', 'ingredientes': 'bacon'}, {'cantidad': '6', 'comida': 'pizzas', 'ingredientes': 'chorizo'}]
(S Quiero/vmip1s0 (Cantidad 3/Z) (Comida pizzas/nc0p000))
[{'cantidad': '3', 'comida': 'pizzas', 'ingredientes': 'null'}]
(S
  Yo/pp1csn00
  quería/vmii1s0
  una/di0fs0
  (Comida ham

In [31]:
score = bigram_chunker.evaluate(listafrasestest)
print(score)

ChunkParse score:
    IOB Accuracy: 100.0%%
    Precision:    100.0%%
    Recall:       100.0%%
    F-Measure:    100.0%%


<b><font color='red'>
    EJERCICIO 3:<br/>
    Analizador de envios de comida con <i>NaiveBayesClassifier</i>
    </font>
</b>

<b>
    Para este ejercicio voy a usar los mismos ficheros de entrenamiento y test que para el anterior.<br/>
    También voy a utilizar la misma función pedir_Comida2() para extraer la información de los chunks.<br/><br/>
    El objetivo es diseñar un chunker como en el ejercicio 2, pero esta vez ,en la fase de
    POS tagging vamos a utilizar un clasificador NaiveBayes.<br/><br/>
    La clase para hacer chunking es análoga a la del ejercicio anterior y se llamará 
    <font color='blue'>ConsecutiveNPChunker</font>, 
    pero esta vez el POS tagger no va a a ser <i>UnigramTagger/BigramTagger</i> sino que usaremos
    una clase llamada <font color='blue'>ConsecutiveNPChunkTagger</font> en cuyo método
    <i>__nit__() habrá un objeto <i>classifier</i> que almacena el clasificador NaiveBayes.
    Esta clase tagger entrenará al clasificador según las etiquetas POS y chunk del archivo de entrenamiento.
    Cuando llamemos al método <i>parse</i> con el corpus de testing se proveerá al clasificador
    de una serie de características que lo asemejan a un BigramTagger:
    <ol>
        <li>La etiqueta POS del token anterior.</li>
        <li>El token actual</li>
        <li>La etiqueta POS del token actual</li>
    </ol>
</b>

In [32]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    return {"pos": pos, "word": word, "prevpos": prevpos}

class ConsecutiveNPChunkTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
    
    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)
    
class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)
            
    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

<b>CHUNKING<br/>
    Ya estan leidos los ficheros con el corpus de entrenamiento y de test del ejercicio 2 y también está
    transformado en nltk.Tree el corpus de entrenamiento.<br/><br/>
    Vamos a entrenar el chunker y llamaremos al método parse de ConsecutiveNPChunker :
    </b>

In [36]:
naivechunker = ConsecutiveNPChunker(train_sents)

In [37]:
listafrasestest = []
for frasetest in tagged_frases:
    arbol = naivechunker.parse(frasetest)
    print(arbol)
    listafrasestest.append(arbol)
    print(pedir_Comida2(arbol))

(S
  Vamos/vmip1p0
  a/sps00
  (Ingredientes querer/vmn0000)
  (Cantidad una/di0fs0)
  (Comida pizza/nc0p000)
  con/sps00
  (Ingredientes orégano/ncfs000)
  ./Fp)
[{'cantidad': 'una', 'comida': 'pizza', 'ingredientes': 'querer'}]
(S
  ¿Podemos/nc00000
  pedir/vmn0000
  (Cantidad 6/Z)
  (Comida hamburguesas/nc0p000)
  con/sps00
  (Ingredientes huevo/ncms000)
  (Ingredientes ?/Fit)
  Gracias/nc00000)
[{'cantidad': '6', 'comida': 'hamburguesas', 'ingredientes': 'huevo'}]
(S
  Me/pp1cs000
  apetecen/nc00000
  (Cantidad 9/Z)
  (Comida bocadillos/nc0p000)
  de/sps00
  (Ingredientes bacon/ncmn000)
  y/cc
  (Cantidad 6/Z)
  (Comida pizzas/nc0p000)
  familiares/ncmp000
  con/sps00
  (Ingredientes chorizo/ncfs000)
  ./Fp)
[{'cantidad': '9', 'comida': 'bocadillos', 'ingredientes': 'bacon'}, {'cantidad': '6', 'comida': 'pizzas', 'ingredientes': 'chorizo'}]
(S Quiero/vmip1s0 (Cantidad 3/Z) (Comida pizzas/nc0p000))
[{'cantidad': '3', 'comida': 'pizzas', 'ingredientes': 'null'}]
(S
  Yo/pp1csn00
  (I

In [38]:
print(naivechunker.evaluate(listafrasestest))

ChunkParse score:
    IOB Accuracy: 100.0%%
    Precision:    100.0%%
    Recall:       100.0%%
    F-Measure:    100.0%%
